In [ ]:
from xml.dom import minidom

## Read XML file

In [ ]:
annot = minidom.parse("./data/annot.annotations") # open XML file

In [ ]:
regions = annot.getElementsByTagName("Region") # list of regions in the annotation file
print(len(regions))

In [ ]:
V = [] # list of vertices in each region
V_coord = [] # list of coord of each vertex in each region
for r in regions :
    vertices = r.getElementsByTagName("V")
    V.append(vertices)
    
    v_coord = []
    for v in vertices :
        coord = {"X": v.getAttribute('X'), "Y": v.getAttribute('Y')}
        v_coord.append(coord)
    V_coord.append(v_coord)

In [ ]:
V_coord

#### 0/1 array representation of regions of interest

In [ ]:
# Images
import os
import PIL 
import numpy as np
import openslide

# Annotations
from xml.dom import minidom
import skimage.draw

In [ ]:
img = np.ones((1008, 840)) # creates an array of ones of the same shape of our image
mask_img = np.zeros((1008, 840, 3), dtype=np.uint8)
ds_rate = 64 # downsample rate

polygons = [] # list of polygons, each object is an array containing the coord of the px in a region of V_coord

for region in V_coord:
    x = [] # list of x coord in the region
    y = [] # list of y coord in the region

    for v in region:
        # as we dont take the full resolution slide of the image, we need to divide the coord of each vertice by the downsampling rate
        temp_x = int(v['X'])/ds_rate 
        temp_y = int(v['Y'])/ds_rate

        x.append(int(temp_x))
        y.append(int(temp_y))

    # sikimage computes the coord of each px in the region from the vertices defining the perimeter of the region
    polygons.append(skimage.draw.polygon(x,y)) 

In [ ]:
def png_mask(img_shape, ds_rate, V_coord):
    mask = np.zeros((img_shape[0], img_shape[1], 3), dtype=np.uint8) # creates an array of ones of the same shape of our image
    #ds_rate = 8 # downsample rate

    # list of polygons, each object is an array containing the coords of the px in a region of V_coord
    polygons = [] # 1 polygon / region

    for region in V_coord:
        x = [] # list of x coord in the region
        y = [] # list of y coord in the region

        # fill the x and y coord arrays
        for v in region:
            # as we dont take the full resolution slide of the image, 
            # we need to divide the coord of each vertice by the downsampling rate
            temp_x = int(v['X'])/ds_rate 
            temp_y = int(v['Y'])/ds_rate

            x.append(int(temp_x))
            y.append(int(temp_y))

        # sikimage computes the coord of each px in the region from the vertices defining 
        # the perimeter of the region
        polygons.append(skimage.draw.polygon(x,y)) 
        
    # modify mask so that each px in a region of V_coord is set to 0
    
    for p in range(len(polygons)):
        poly = np.transpose(polygons[p])

        for i in range(len(poly)):
            mask[(poly[i][1], poly[i][0])] = [255, 255, 255]
            
       
    return mask

In [ ]:
import pipeline_IHC as pipe
directory = "./test_data/test_png/"
image = pipe.load_img(directory)
name = "CRC1"
xml = minidom.parse("./test_data/test_png/CRC1.annotations")
coords = pipe.xml_to_vertices(xml, '0')
img_shape = image[name].shape
print(img_shape)
mask_img = png_mask(img_shape, 32, coords)

In [ ]:
PILimg = PIL.Image.fromarray(mask_img, 'RGB')
PILimg.save('mask.png')
PILimg.show()

In [ ]:
# modify img so that each px in a region of V_coord is set to 0
for p in range(len(polygons)):
    poly = np.transpose(polygons[p])

    for i in range(len(poly)):
        mask_img[(poly[i][0], poly[i][1])] = [255, 255, 255]
        img[(poly[i][0], poly[i][1])] = 0

In [ ]:
mask_img.shape

In [ ]:
PILimg = PIL.Image.fromarray(mask_img, 'RGB')
PILimg.save('my.png')
PILimg.show()

## Write XML file

In [ ]:
new_annot = minidom.Document() # create a new file

annotations = new_annot.createElement("Annotations") # create element Annotations

annotation = new_annot.createElement("Annotation") # create element Annotation
# set all the attributes 
annotation.setAttribute('LineColor',"65535") 
annotation.setAttribute('Name', "Layer 1")
annotation.setAttribute('Visible', "True")

annotations.appendChild(annotation) # add Annotation to Annotations

In [ ]:
for R in V_coord:
    new_region = new_annot.createElement("Region") # create a new region
    for V in R:
        new_vertice = new_annot.createElement("V") # create a new vertice
        # set vertice's coord
        new_vertice.setAttribute('X', V['X'])
        new_vertice.setAttribute('Y', V['Y'])
        
        new_region.appendChild(new_vertice) # add vertice to region
        
    annotation.appendChild(new_region) # add region to annotation

In [ ]:
new_annot.appendChild(annotations) # add Annotations to our document

In [ ]:
myfile = open("./data/annot_new.annotations", "w") # open a new file
myfile.write(new_annot.toprettyxml()) # write the content of our document in XML in the file
myfile.close()